<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/RMSD_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script RMSD with Savitzky–Golay smoothing (using GROMACS outputs)**

In [ ]:
# ================================================
# MDAnalysis Analysis Script
# RMSD + RMSF + H-Bonds + Radius of Gyration + SASA
# With Savitzky–Golay smoothing
# ================================================

In [ ]:
# --- Install MDAnalysis ---
!pip install MDAnalysis MDAnalysisData

In [ ]:
# --- Import libraries ---
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import MDAnalysis as mda
from MDAnalysis.analysis import rms, hbonds, polymer, sasa

In [ ]:
# ===== Settings =====
SMOOTH_WINDOW = 11   # maximum smoothing window size (must be odd)
POLY_ORDER = 2       # polynomial order for Savitzky–Golay
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
FIGSIZE = (11, 6)
DPI = 1200
# Legend names
LEGEND_NAMES = ["Run 1", "Run 2", "Run 3"]
# ====================


In [ ]:
# === Input files (edit with your runs) ===
# Each run must have topology (.tpr/.gro/.pdb) + trajectory (.xtc/.trr)
runs = [
    ("run1.tpr", "run1.xtc"),
    ("run2.tpr", "run2.xtc"),
    ("run3.tpr", "run3.xtc"),
]



In [ ]:
# --- Helper: smoothing with safe window ---
def smooth_curve(data, window=SMOOTH_WINDOW, poly=POLY_ORDER):
    n = len(data)
    if n < 3:
        return data.copy()
    w = min(window, n)
    if w % 2 == 0:
        w = max(3, w - 1)
    if w > poly:
        return savgol_filter(data, window_length=w, polyorder=poly, mode="interp")
    return data.copy()


In [ ]:
# ========================
# RMSD
# ========================
plt.figure(figsize=FIGSIZE)
all_rmsd_values = []

for i, (tpr, xtc) in enumerate(runs):
    print(f"\n=== RMSD: {tpr} + {xtc} ===")
    u = mda.Universe(tpr, xtc)
    ref = mda.Universe(tpr)

    # Calculate RMSD
    R = rms.RMSD(u, ref, select="protein and name CA")
    R.run()

    time = R.results.rmsd[:, 1] / 1000.0
    rmsd = R.results.rmsd[:, 2]
    all_rmsd_values.extend(rmsd)

    smooth = smooth_curve(rmsd)
    mean_val, std_val = np.mean(rmsd), np.std(rmsd)

    color = COLORS[i % len(COLORS)]
    curve_name = LEGEND_NAMES[i] if i < len(LEGEND_NAMES) else f"Run {i+1}"

    plt.plot(time, rmsd, linewidth=1.0, alpha=0.15, color=color)
    plt.plot(time, smooth, label=f"{curve_name}", color=color)
    plt.fill_between(time, smooth - std_val, smooth + std_val,
                     alpha=0.05, color=color)

    print(f"{curve_name}: Mean = {mean_val:.3f} nm | Std = {std_val:.3f} nm")

# Global stats
all_rmsd_values = np.array(all_rmsd_values)
print("\n=== RMSD Global Statistics ===")
print(f"Mean = {np.mean(all_rmsd_values):.4f} nm | Std = {np.std(all_rmsd_values):.4f} nm")

plt.xlabel("Time (ns)")
plt.ylabel("RMSD (nm)")
plt.xlim(time.min(), time.max())
plt.ylim(0.0, 1.5)
plt.legend()
plt.tight_layout()
plt.savefig("rmsd.png", dpi=DPI, bbox_inches="tight")
plt.close()


In [ ]:
# ========================
# RMSF
# ========================
for i, (tpr, xtc) in enumerate(runs):
    print(f"\n=== RMSF: {tpr} + {xtc} ===")
    u = mda.Universe(tpr, xtc)
    protein = u.select_atoms("protein and name CA")

    rmsf = rms.RMSF(protein).run()
    resids = protein.resids
    values = rmsf.results.rmsf

    plt.figure(figsize=FIGSIZE)
    plt.plot(resids, values, label=LEGEND_NAMES[i],
             color=COLORS[i % len(COLORS)])
    plt.xlabel("Residue ID")
    plt.ylabel("RMSF (nm)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"rmsf_run{i+1}.png", dpi=DPI, bbox_inches="tight")
    plt.close()


In [ ]:
# ========================
# HBonds
# ========================
for i, (tpr, xtc) in enumerate(runs):
    print(f"\n=== HBonds: {tpr} + {xtc} ===")
    u = mda.Universe(tpr, xtc)
    h = hbonds.HydrogenBondAnalysis(u, selection1="protein", selection2="protein")
    h.run()

    times = h.results.times / 1000.0
    counts = h.results.count_by_time

    smooth = smooth_curve(counts)

    plt.figure(figsize=FIGSIZE)
    plt.plot(times, counts, alpha=0.15, color=COLORS[i % len(COLORS)])
    plt.plot(times, smooth, label=LEGEND_NAMES[i],
             color=COLORS[i % len(COLORS)])
    plt.xlabel("Time (ns)")
    plt.ylabel("# H-Bonds")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"hbonds_run{i+1}.png", dpi=DPI, bbox_inches="tight")
    plt.close()


In [ ]:
# ========================
# Radius of Gyration
# ========================
for i, (tpr, xtc) in enumerate(runs):
    print(f"\n=== Radius of Gyration: {tpr} + {xtc} ===")
    u = mda.Universe(tpr, xtc)
    protein = u.select_atoms("protein")

    Rg = polymer.RadiusGyration(protein).run()
    times = Rg.results.times / 1000.0
    rg = Rg.results.rg

    smooth = smooth_curve(rg)

    plt.figure(figsize=FIGSIZE)
    plt.plot(times, rg, alpha=0.15, color=COLORS[i % len(COLORS)])
    plt.plot(times, smooth, label=LEGEND_NAMES[i],
             color=COLORS[i % len(COLORS)])
    plt.xlabel("Time (ns)")
    plt.ylabel("Rg (nm)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"rg_run{i+1}.png", dpi=DPI, bbox_inches="tight")
    plt.close()



In [ ]:
# ========================
# SASA
# ========================
for i, (tpr, xtc) in enumerate(runs):
    print(f"\n=== SASA: {tpr} + {xtc} ===")
    u = mda.Universe(tpr, xtc)
    protein = u.select_atoms("protein")

    s = sasa.SASA(protein, probe_radius=0.14)
    s.run()

    sasa_values = s.results.sasa
    times = np.arange(len(sasa_values)) * u.trajectory.dt / 1000.0

    smooth = smooth_curve(sasa_values)

    plt.figure(figsize=FIGSIZE)
    plt.plot(times, sasa_values, alpha=0.15, color=COLORS[i % len(COLORS)])
    plt.plot(times, smooth, label=LEGEND_NAMES[i],
             color=COLORS[i % len(COLORS)])
    plt.xlabel("Time (ns)")
    plt.ylabel("SASA (nm²)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"sasa_run{i+1}.png", dpi=DPI, bbox_inches="tight")
    plt.close()

In [ ]:
# --- Import image files ---
from google.colab import files

files.download("rmsd.png")
files.download("rmsf_run1.png")
files.download("hbonds_run1.png")
files.download("rg_run1.png")
files.download("sasa_run1.png")